In [107]:
import pandas as pd
import numpy as np
import os
import pickle

In [ ]:
with open('exper_status.dict', 'rb') as f:
    exper_status = pickle.load(f)

## Load results 

In [118]:
with open('results_summary.dict', 'rb') as f:
    stats = pickle.load(f)

In [119]:
stats

{'T2F': {'baseline_nn': {'valid_time': (1.3893952369689941, 0.0),
   'valid_steps': (10.0, 0.0),
   'valid_loss': (15.3416839273927, 0.0),
   'valid_eval': (-22.76826207417731, 0.0),
   'valid_dist': (9.815769779238001e-06, 0.0),
   'valid_ineq_max': (0.032298444922559164, 0.0),
   'valid_ineq_mean': (0.0014462382263729936, 0.0),
   'valid_ineq_num_viol_0': (4198.741935483871, 0.0),
   'valid_ineq_num_viol_1': (3680.8548387096776, 0.0),
   'valid_ineq_num_viol_2': (141.88709677419354, 0.0),
   'valid_eq_max': (0.05094687977025511, 0.0),
   'valid_eq_mean': (0.0044429437392230564, 0.0),
   'valid_eq_num_viol_0': (5649.0, 0.0),
   'valid_eq_num_viol_1': (4942.0, 0.0),
   'valid_eq_num_viol_2': (417.0, 0.0),
   'valid_raw_time': (0.0009984970092773438, 0.0),
   'valid_raw_eval': (-22.768434169347554, 0.0),
   'valid_raw_ineq_max': (0.03229174933803204, 0.0),
   'valid_raw_ineq_mean': (0.0014462472076048954, 0.0),
   'valid_raw_ineq_num_viol_0': (4197.806451612903, 0.0),
   'valid_raw_ineq

## Main tables

In [110]:
def get_latex(df):
    print(df.to_latex().replace('\$', '$'))

In [111]:
def get_results_table(results_dict, metrics, keep_methods):
    d = {}
    missing_methods = []
    for method in keep_methods:
        if method in results_dict:
            d[method] = ['{} ({})'.format(*results_dict[method][metric]) if 'time' in metric else \
                         '{} ({})'.format(*results_dict[method][metric]) for metric in metrics]
        else:
            missing_methods.append(method)
    df = pd.DataFrame.from_dict(d, orient='index')
    # df.columns = metrics
    df.index.names = ['alg']
    if len(missing_methods) > 0:
        print('missing methods: {}'.format(missing_methods))
    return df.loc[[x for x in keep_methods if x not in missing_methods]], missing_methods

In [112]:
def get_full_results_table(results_dict, exper_type, num=5):
    metrics = ['test_eval', 
               'test_eq_max', 'test_eq_mean', 
               'test_ineq_max', 'test_ineq_mean',
               'test_time']
    metrics_raw = [x.replace('test_', 'test_raw_') for x in metrics]
    metrics_renaming = ['Obj. value', 
                        'Max eq viol.', 'Mean eq viol.', 
                        'Max ineq viol.', 'Mean ineq viol.',
                        'Time']
    
    nn_methods = ['method', 'method_no_compl', 'method_no_corr', 'method_no_soft', 
               'baseline_nn', 'baseline_eq_nn']
    opt_methods = dict([
         ('T2F',['osqp'])
    ])
    opt_methods_list = ['baseline_opt_{}'.format(x) for x in opt_methods[exper_type]]
    methods_renaming_dict = dict((
            ('method', 'DC3'),
            ('method_no_compl', 'DC3, $\neq$'),
            ('method_no_corr', 'DC3, $\not\leq$ train'),
            ('method_no_corr-noTestCorr', 'DC3, $\not\leq$ train/test'),
            ('method_no_soft', 'DC3, no soft loss'),
            ('baseline_nn-noTestCorr', 'NN'),
            ('baseline_nn', 'NN, $\leq$ test'),
            ('baseline_eq_nn-noTestCorr', 'Eq.~NN'),
            ('baseline_eq_nn', 'Eq.~NN, $\leq$ test'),
            ('baseline_opt_osqp', 'Optimizer (OSQP)'),
            ('baseline_opt_qpth', 'Optimizer (qpth)'),
            ('baseline_opt_ipopt', 'Optimizer (IPOPT)'),
            ('baseline_opt_pypower', 'Optimizer (PYPOWER)')
        ))
    
    df1, missing_methods = get_results_table(results_dict, metrics, nn_methods + opt_methods_list)
    
    df2, missing_methods_2 = get_results_table(results_dict, metrics_raw, nn_methods)
    # df2.columns = metrics
    df2.index =  ['{}-noTestCorr'.format(x) for x in df2.index]
    
    missing_methods = missing_methods + ['{}-noTestCorr'.format(x) for x in missing_methods_2]
        
    all_results = pd.concat([df1, df2])
    results_ordering = opt_methods_list + ['method', 'method_no_compl', 'method_no_corr', 'method_no_corr-noTestCorr',
                       'method_no_soft', 'baseline_nn-noTestCorr', 'baseline_nn', 'baseline_eq_nn-noTestCorr', 
                       'baseline_eq_nn']
    all_results = all_results.loc[[x for x in results_ordering if x not in missing_methods]]
    all_results.index = [methods_renaming_dict[x] for x in all_results.index]
    all_results.columns = metrics_renaming
    
    return all_results

In [113]:
df_simple_5050 = get_full_results_table(stats['T2F'], exper_type='T2F')
df_simple_5050

missing methods: ['method', 'method_no_compl', 'method_no_corr', 'method_no_soft', 'baseline_eq_nn']
missing methods: ['method', 'method_no_compl', 'method_no_corr', 'method_no_soft', 'baseline_eq_nn']


,Obj. value,Max eq viol.,Mean eq viol.,Max ineq viol.,Mean ineq viol.,Time
Optimizer (OSQP),-0.006183655878068966 (0.0),2.8005323105756025e-05 (0.0),6.763315110870361e-07 (0.0),0.00031431917475913834 (0.0),6.442036467223166e-07 (0.0),0.01578302921787385 (0.0)
NN,-22.768434169347554 (0.0),0.051306227784726585 (0.0),0.004443335301471124 (0.0),0.03203963653368255 (0.0),0.0014307706521455722 (0.0),0.0 (0.0)
"NN, $\leq$ test",-22.768262079391498 (0.0),0.050946898107745206 (0.0),0.0044429437426995125 (0.0),0.032046456513379874 (0.0),0.0014307617971874129 (0.0),1.3873157501220703 (0.0)


## Appendix tables (simple problem)

In [114]:
def get_comparison_table(all_stats, experiments, metrics, keep_methods):
    d = {}
    for method in keep_methods:
        for metric in metrics:
            d[(method, metric)] = {}
            for experiment in experiments:
                d[(method, metric)][experiment] = '{:.2f} ({:.2f})'.format(*all_stats[experiment][method][metric])
        df = pd.DataFrame.from_dict(d, orient='index')           
    df.index.names = ['Alg', 'Metric']
    return df.iloc[df.index.isin(keep_methods, 'Alg')]

In [115]:
def get_simple_comparison_table(all_stats, vary_across):
    assert vary_across in ['ineq', 'eq'], 'vary_across should be in [ineq, eq]'
    
    if vary_across == 'ineq':
        experiments = ['simple_ineq10_eq50', 'simple_ineq30_eq50', 'simple_ineq50_eq50', 'simple_ineq70_eq50',
                       'simple_ineq90_eq50']
        experiments_renaming = ['10', '30', '50', '70', '90']
    else:
        experiments = ['simple_ineq50_eq10', 'simple_ineq50_eq30', 'simple_ineq50_eq50', 'simple_ineq50_eq70',
                       'simple_ineq50_eq90']
        experiments_renaming = ['10', '30', '50', '70', '90']
    
    methods = ['method', 'method_no_compl', 'method_no_corr', 'method_no_soft', 
               'baseline_nn', 'baseline_eq_nn', 'baseline_opt_osqp']
    methods_renaming_dict = dict((
            ('method', 'DC3'),
            ('method_no_compl', 'DC3, $\neq$'),
            ('method_no_corr', 'DC3, $\not\leq$ train'),
            ('method_no_corr-noTestCorr', 'DC3, $\not\leq$ train/test'),
            ('method_no_soft', 'DC3, no soft loss'),
            ('baseline_nn-noTestCorr', 'NN'),
            ('baseline_nn', 'NN, $\leq$ test'),
            ('baseline_eq_nn-noTestCorr', 'Eq.~NN'),
            ('baseline_eq_nn', 'Eq.~NN, $\leq$ test'),
            ('baseline_opt_osqp', 'Optimizers (OSQP, qpth)')
        ))
    
    metrics = ['test_eval', 'test_eq_max', 'test_ineq_max']
    metrics_renaming_dict = dict((('test_eval', 'Obj. val.'), ('test_eq_max', 'Max eq.'), ('test_ineq_max', 'Max ineq.')  ))
    clean_metrics = [metrics_renaming_dict[x] for x in metrics]
    metrics_raw = [x.replace('test_', 'test_raw_') if 'time' not in x else x for x in metrics]
    
    df1 = get_comparison_table(all_stats, experiments, metrics, methods)
    df1 = df1.reset_index()
    df1['Metric'] = df1['Metric'].str.replace('_obj_val', '_eval') # make naming consistent
    df1 = df1.set_index(['Alg', 'Metric'])
    
    df2 = get_comparison_table(all_stats, experiments, metrics_raw, methods[:-1])
    df2.columns = experiments
    df2 = df2.reset_index()
    df2['Alg'] = ['{}-noTestCorr'.format(x) for x in df2['Alg']]
    df2['Metric'] = df2['Metric'].str.replace('raw_', '')  # make naming consistent
    df2 = df2.set_index(['Alg', 'Metric'])
    
    all_results = pd.concat([df1, df2])
    results_ordering = ['baseline_opt_osqp', 'method', 'method_no_compl', 'method_no_corr', 'method_no_corr-noTestCorr',
                       'method_no_soft', 'baseline_nn-noTestCorr', 'baseline_nn', 'baseline_eq_nn-noTestCorr', 
                       'baseline_eq_nn']
    clean_results_ordering = [methods_renaming_dict[x] for x in results_ordering]
    all_results = all_results.iloc[all_results.index.isin(results_ordering, 'Alg')]
    all_results = all_results.reindex(pd.MultiIndex.from_product([results_ordering, metrics], names=['Alg', 'Metric']))
    all_results = all_results.reset_index()
    all_results['Alg'] = [methods_renaming_dict[x] for x in all_results['Alg']]
    all_results['Metric'] = [metrics_renaming_dict[x] for x in all_results['Metric']]
    all_results = all_results.set_index(['Alg', 'Metric'])
    all_results.columns = experiments_renaming
    all_results = all_results.reindex(pd.MultiIndex.from_product([clean_results_ordering, clean_metrics])) # remove names
    
    return all_results

In [116]:
df_simple_varyeq = get_simple_comparison_table(stats, 'eq')
df_simple_varyeq

KeyError: 'simple_ineq50_eq10'

In [ ]:
df_simple_varyineq = get_simple_comparison_table(stats, 'ineq')
df_simple_varyineq

KeyError: 'simple_ineq10_eq50'

## LaTeX 

In [ ]:
get_latex(df_simple_5050)

\begin{tabular}{lllllll}
\toprule
{} &                   Obj. value &                  Max eq viol. &                Mean eq viol. &                Max ineq viol. &              Mean ineq viol. &                       Time \\
\midrule
Optimizer (OSQP) &  -0.006183655878068966 (0.0) &  2.8005323105756025e-05 (0.0) &  6.763315110870361e-07 (0.0) &  0.00031431917475913834 (0.0) &  6.442036467223166e-07 (0.0) &  0.01578302921787385 (0.0) \\
\bottomrule
\end{tabular}



C:\Users\HP OMEN\AppData\Local\Temp\ipykernel_28720\761946459.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex().replace('\$', '$'))


In [ ]:
get_latex(df_simple_varyeq)

In [ ]:
get_latex(df_simple_varyineq)